In [1]:
!pip install ctransformers[cuda]
!pip install --upgrade git+https://github.com/huggingface/transformers

zsh:1: no matches found: ctransformers[cuda]
  Cloning https://github.com/huggingface/transformers to /private/var/folders/bl/kzfk5ts90gj98y9jcv49ynmc0000gn/T/pip-req-build-_9oh9s_4
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /private/var/folders/bl/kzfk5ts90gj98y9jcv49ynmc0000gn/T/pip-req-build-_9oh9s_4
  Resolved https://github.com/huggingface/transformers to commit 0290ec19c901adc0f1230ebdccad11c40af026f5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done


In [10]:
!pip install einops

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


zsh:1: no matches found: bertopic[spacy]


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [1]:
from ctransformers import AutoModelForCausalLM
from transformers import AutoTokenizer, pipeline

# Set gpu_layers to the number of layers to offload to GPU. Set to 0 if no GPU acceleration is available on your system.
model = AutoModelForCausalLM.from_pretrained(
    "TheBloke/zephyr-7B-alpha-GGUF",
    model_file="zephyr-7b-alpha.Q4_K_M.gguf",
    model_type="mistral",
    gpu_layers=0,
    hf=True
)
tokenizer = AutoTokenizer.from_pretrained("avsolatorio/GIST-Embedding-v0")

# Pipeline
generator = pipeline(
    model=model, tokenizer=tokenizer,
    task='text-generation',
    max_new_tokens=50,
    repetition_penalty=1.1
)

/Users/ayushjain/Development/DisruptionLab/jinship_sp2024/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/ayushjain/Development/DisruptionLab/jinship_sp2024/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 1 files: 100%|██████████| 1/1 [00:00<00:00, 16912.52it/s]


In [2]:
prompt = """<|system|>You are a helpful, respectful and honest assistant for labeling topics..</s>
<|user|>
I have a topic that contains the following documents:
[DOCUMENTS]

The topic is described by the following keywords: '[KEYWORDS]'.

Based on the information about the topic above, please create a short label of this topic. Make sure you to only return the label and nothing more.</s>
<|assistant|>"""


In [3]:
from datasets import load_dataset
from transformers import BertTokenizer
import numpy as np


dataset = load_dataset("valurank/Topic_Classification")["train"]
# Extract abstracts to train on and corresponding titles
descriptions = dataset["article_text"]
topics = dataset["topic"]

In [12]:
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk

nltk.download('punkt')

sent_map = {}
sentences = []

for desc in descriptions[:10]:
    if isinstance(desc, str):
        for sentence in sent_tokenize(desc):
            sentences.append(sentence)
            sent_map[sentence] = desc

[nltk_data] Downloading package punkt to /Users/ayushjain/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [13]:
from bertopic.representation import TextGeneration
from bertopic import BERTopic

# Text generation with Zephyr
zephyr = TextGeneration(generator, prompt=prompt)

In [14]:
from bertopic.representation import KeyBERTInspired, MaximalMarginalRelevance, OpenAI, PartOfSpeech

# KeyBERT
keybert_model = KeyBERTInspired()

# # Part-of-Speech
# pos_model = PartOfSpeech("en_core_web_sm")

# MMR
mmr_model = MaximalMarginalRelevance(diversity=0.3)

In [15]:
representation_model = {
  "Zephyr": zephyr,
  "KeyBert": keybert_model,
  "MMR": mmr_model
}

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer_model = CountVectorizer(stop_words="english", min_df=2, ngram_range=(1, 2))

In [17]:
from sentence_transformers import SentenceTransformer

e_model = SentenceTransformer("avsolatorio/GIST-Embedding-v0")
embeddings = e_model.encode(sentences, show_progress_bar=True)

Batches: 100%|██████████| 10/10 [00:02<00:00,  3.97it/s]


In [18]:
topic_model = BERTopic(representation_model=representation_model, vectorizer_model=vectorizer_model, verbose=True)

In [19]:
topic_model.fit(sentences, embeddings=embeddings)

2024-03-12 12:17:26,630 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-03-12 12:17:27,894 - BERTopic - Dimensionality - Completed ✓
2024-03-12 12:17:27,894 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-03-12 12:17:27,900 - BERTopic - Cluster - Completed ✓
2024-03-12 12:17:27,901 - BERTopic - Representation - Extracting topics from clusters using representation models.
 40%|████      | 4/10 [00:52<01:18, 13.03s/it]Token indices sequence length is longer than the specified maximum sequence length for this model (513 > 512). Running this sequence through the model will result in indexing errors
Number of tokens (513) exceeded maximum context length (512).
100%|██████████| 10/10 [03:16<00:00, 19.68s/it]


AttributeError: 'NoneType' object has no attribute 'embed_documents'

In [20]:
topic_distr, _ = topic_model.approximate_distribution(sentences)

100%|██████████| 1/1 [00:00<00:00, 28.49it/s]


In [21]:
topic_table = topic_model.get_topic_info()

TypeError: 'NoneType' object is not callable

In [15]:
topic_distr[0]  

array([0., 1., 0., 0., 0., 0., 0., 0.])

In [16]:
topic_table['Zephyr']

0    [\n"Lloyd-Webber's Final Letter and Boos at Ci...
1    [\n"Jurassic World: Dominion - Critical Analys...
2    [\n"News Listening & Following Options", , , ,...
3    [\n"Gaia's New Data Reveals Orbit Trails of As...
4    [\nAngels' Trout discusses fantasy football co...
5    [\nTopic Label |>\nTopic: Topic>\nTopic: The t...
6    [\nTopic: Comparison>\nLabeling: Topic>\nTopic...
7    [\nLooney's Insights on Curry & Thompson Shoot...
8    [\nTopic: Gaming>\n- The topic |>\n- Bethes|>\...
Name: Zephyr, dtype: object

In [ ]:
summary_tags = {desc: [] for desc in descriptions}

for s_idx, sent_distr in enumerate(topic_distr):
  for i, prob in enumerate(sent_distr):
    if prob > 0.5 or prob == max(sent_distr):
      summary_tags[sent_map[sentences[s_idx]]].append(topic_table['Zephyr'][i][0])

In [ ]:
for desc, tags in summary_tags.items():
    first_few_words = ' '.join(desc.split()[:5])  # Adjust the number as needed
    print(f"Description: {first_few_words}...\nTags: {', '.join(tags)}\n")

Description: NEWYou can now listen to...
Tags: 
Topic Label |>
Topic: Topic>
Topic: The topic |>
Topic: The topic |>
Topic: The topic |>
Topic: The topic |>
Topic: The topic |>
Topic: The, 
Topic Label |>
Topic: Topic>
Topic: The topic |>
Topic: The topic |>
Topic: The topic |>
Topic: The topic |>
Topic: The topic |>
Topic: The, 
Topic Label |>
Topic: Topic>
Topic: The topic |>
Topic: The topic |>
Topic: The topic |>
Topic: The topic |>
Topic: The topic |>
Topic: The, 
Topic Label |>
Topic: Topic>
Topic: The topic |>
Topic: The topic |>
Topic: The topic |>
Topic: The topic |>
Topic: The topic |>
Topic: The, 
"Jurassic World: Dominion - Critical Analysis", 
Topic Label |>
Topic: Topic>
Topic: The topic |>
Topic: The topic |>
Topic: The topic |>
Topic: The topic |>
Topic: The topic |>
Topic: The, 
Topic Label |>
Topic: Topic>
Topic: The topic |>
Topic: The topic |>
Topic: The topic |>
Topic: The topic |>
Topic: The topic |>
Topic: The, 
Topic Label |>
Topic: Topic>
Topic: The topic |>
To